In [ ]:
get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')
from commercial_search import *
from commercial_check import *
from compute_histogram import *
# import sys
# default_stdout = sys.stdout
# sys.stdout = open('../log/log_commercial_search.txt', 'w')

# Prepare data

In [ ]:
# compute histogram
video_list_path = '../data/video_list_Rachel_Maddow.txt'
prepare_black_frame_list_multithread(video_list_path, 16)
# prepare_black_frame_list(video_list_path)

In [ ]:
# reformatting dicts from Will

# video_meta_list = pickle.load(open('../data/video_meta.pkl', 'rb'))
# video_meta_dict = {}
# for item in video_meta_list:
#     video_name = item['path'].replace('tvnews/videos/', '').replace('.mp4', '')
#     fps = item['fps']
#     video_frames = item['num_frames']
#     video_length = int(video_frames / fps)
#     video_meta_dict[video_name] = {'fps': fps, 'video_frames': video_frames, 'video_length': video_length}
# pickle.dump(video_meta_dict, open('../data/video_meta_dict.pkl', 'wb'))

all_shots = pickle.load(open('../data/all_shots.pkl', 'rb'))
# print(len(shot_dict))
out = open('../data/total_video_list.txt', 'w')
black_frame_dict = {}
shot_dict = {}
for path in sorted(all_shots):
    video_name = path.replace('tvnews/videos/', '').replace('.mp4', '')
    shot_dict[video_name] = all_shots[path][0]
    black_frame_dict[video_name] = all_shots[path][1]
    out.write(video_name + '\n')
out.close()
pickle.dump(shot_dict, open('../data/shot_dict.pkl', 'wb'))
pickle.dump(black_frame_dict, open('../data/black_frame_dict.pkl', 'wb'))

In [ ]:
# prepare video list for unsupervised test
outfile = open('../data/video_list_Wolf.txt', 'w')
video_list = open('../data/total_video_list.txt', 'r').read().split('\n')

# CNN = ['CNN_Newsroom_Live.mp4', 'New_Day.mp4']
CNN = ["Wolf"]
# FOX = ['Happening_Now', 'Hannity']
FOX = []
# MSNBC = ['MSNBC_Live.mp4', 'The_Rachel_Maddow_Show']
MSNBC = []

select_video = []
for year in range(2015,2018):
    for month in range(1, 13):
        for day in ['05', '10', '15', '25']:
            date = str(year) + '{:02d}'.format(month) + day
            found = []
            for video in video_list:
                if video.find(date) != -1:
                    for cnn in CNN:
                        if not cnn in found and video.find(cnn) != -1:
                            print(video)
                            select_video.append(video)
                            found.append(cnn)
                    for fox in FOX:
                        if not fox in found and video.find(fox) != -1:
                            print(video)
                            select_video.append(video)
                            found.append(fox)
                    for msnbc in MSNBC:
                        if not msnbc in found and video.find(msnbc) != -1:
                            print(video)
                            select_video.append(video)
                            found.append(msnbc)

select_video.sort()
for video in select_video:
    outfile.write(video+'\n')
outfile.close()

# detect commercial

In [ ]:
# test on single video
test_single_video("FOXNEWSW_20160527_040000_The_Kelly_File", local=False)

In [ ]:
# test on video list
start_time = time.time()

video_list_path = '../data/labeled_video_list.txt'

result = test_video_list(video_list_path)
print("--- %s seconds ---" % (time.time() - start_time))        

In [ ]:
# detect commercial through multithread
start_time = time.time()

video_list_path = '../data/total_video_list.txt'

search_commercial_multithread(video_list_path, "../data/commercial_dict_raw.pkl", 30)
print("--- %s seconds ---" % (time.time() - start_time))        

# collect tmp result

In [ ]:
commercial_dict_list = []
for i in range(30):
    commercial_dict_list.append('../tmp/commercial_dict_' + str(i) + '.pkl')
    
commercial_dict = pickle.load(open('../data/commercial_dict_raw.pkl','rb'))
for path in commercial_dict_list:
        dict_file = Path(path)
        if not dict_file.is_file():
            continue
        commercial_dict_tmp = pickle.load(open(path, "rb" ))
        commercial_dict = {**commercial_dict, **commercial_dict_tmp}
        
pickle.dump(commercial_dict, open('../data/commercial_dict_raw2.pkl', "wb" ))

# Post process result

In [ ]:
# remove not confident commercial detection results
post_process_result()

# Test API

In [ ]:
import pickle
from detect_commercial import detect_commercial
meta_dict = pickle.load(open("../data/video_meta_dict.pkl", 'rb'))
black_frame_dict = pickle.load(open("../data/black_frame_dict.pkl", 'rb'))

In [ ]:
video_name = 'CNNW_20150805_170000_Wolf'
transcript_path = "../data/videos/" + video_name + '.cc5.srt'
commercial_list, err_str = detect_commercial(meta_dict[video_name], black_frame_dict[video_name], transcript_path)
print(commercial_list, err_str)

In [ ]:
import os
from commercial_search import load_commercial_groundtruth
from utility import get_time_from_second
commercial_gt = load_commercial_groundtruth()
commercial_dict = {}
for video_name in sorted(commercial_gt):
    print(video_name)
    transcript_path = "../data/transcripts_align/" + video_name + '.align.srt'
    if not video_name in meta_dict or not video_name in black_frame_dict or not os.path.exists(transcript_path):
        continue
    commercial_list, err_str = detect_commercial(meta_dict[video_name], black_frame_dict[video_name], transcript_path)
    if not commercial_list is None:
        commercial_dict[video_name] = [((0, get_time_from_second(int(s))),(0, get_time_from_second(e))) for (s,e) in commercial_list]
pickle.dump(commercial_dict, open('../data/commercial_dict_align.pkl', 'wb'))

In [ ]:
from commercial_search import test_all_result
test_all_result('../data/commercial_dict_new.pkl')

In [ ]:
from commercial_search import test_all_result
test_all_result('../data/commercial_dict_align.pkl')